In [3]:
# install plotly and dash, if not yet already
! pip install plotly dash

import plotly, dash

In [14]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

data = pd.read_csv('Task5_1P.csv')

app = Dash()

app.layout = html.Div([
    dcc.Dropdown(
        id='axis-selection',
        options=[
            {'label': 'X', 'value': 'x'},
            {'label': 'Y', 'value': 'y'},
            {'label': 'Z', 'value': 'z'},
            {'label': 'All (X, Y, Z)', 'value': 'all'}
        ],
        value='x'
    ),
    dcc.Dropdown(
        id='graph-type',
        options=[
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Line Chart', 'value': 'line'}
        ],
        value='scatter'
    ),
    dcc.Input(id='num-samples', type='number', value=100),
    html.Button('Previous', id='prev-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),
    dcc.Graph(id='gyroscope-graph'),
    html.Div(id='summary-table')
])

@app.callback(
    Output('gyroscope-graph', 'figure'),
    Output('summary-table', 'children'),
    Input('axis-selection', 'value'),
    Input('graph-type', 'value'),
    Input('num-samples', 'value'),
    Input('prev-button', 'n_clicks'),
    Input('next-button', 'n_clicks')
)

def update_graph(axis_selection, graph_type, num_samples, prev_clicks, next_clicks):
    max_samples = len(data)
    start_index = max(0, (prev_clicks - next_clicks) * num_samples)
    end_index = min(start_index + num_samples, max_samples)
    filtered_data = data.iloc[start_index:end_index]

    # Create the graph
    fig = go.Figure()
    if axis_selection == 'all':
        if graph_type == 'scatter':
            mode = 'markers'
        elif graph_type == 'line':
            mode = 'lines'

        fig.add_trace(go.Scatter(x=data.index, y=data['x'], mode=mode, name='X'))
        fig.add_trace(go.Scatter(x=data.index, y=data['y'], mode=mode, name='Y'))
        fig.add_trace(go.Scatter(x=data.index, y=data['z'], mode=mode, name='Z'))
    else:
        if graph_type == 'scatter':
            fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data[axis_selection], mode='markers'))
        elif graph_type == 'line':
            fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data[axis_selection], mode='lines'))

    # Create the statistical summary table
    selected_data = filtered_data[[axis_selection]] if axis_selection != 'all' else filtered_data[['x', 'y', 'z']]
    summary = selected_data.describe().reset_index()
    summary_table = html.Table([
        html.Tr([html.Th(col) for col in summary.columns])
    ] + [
        html.Tr([html.Td(val) for val in row]) for row in summary.itertuples(index=False)
    ])

    return fig, summary_table

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode = "tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>